In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
import numpy as np

parent = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(parent, "drp1_mito_tracker"))

from extract import tif_to_array, czi_to_array
from display.dashboard import show
from display import colors

In [ ]:
dir_name = "TODO" # SELECT DIRECTORY HERE
filename_suffix = "czi"

In [ ]:
files = []
number = 0
for (dirpath, dirnames, filenames) in os.walk(dir_name):
    for s in filenames:
        if s.endswith('.' + filename_suffix):
            files.append(s)
            print(number, s)
            number += 1
    break

In [ ]:
selected_file = 0 # SELECT FILE HERE
base_filename = files[selected_file][:-4]

mtdnatrac_path = os.path.join(dir_name, base_filename + "_mtdnatrac" + '.csv')
drp1trac_path = os.path.join(dir_name, base_filename + "_drp1trac" + '.csv')
mitoskel_path = os.path.join(dir_name, base_filename + "_mitoskel" + '.csv')

mtdna_df = pd.read_csv(mtdnatrac_path)
drp1_df = pd.read_csv(drp1trac_path)
mito_df = pd.read_csv(mitoskel_path)

In [ ]:
path = os.path.join(dir_name, base_filename + '.' + filename_suffix)
print(path)

data = czi_to_array(path)

mtdna = data['mtdna']
drp1 = data['drp1']
mito = data['mito']

In [ ]:
drp1_cond = ((abs(abs(drp1_df.perp_angle) - 90) < 15) | (drp1_df.d < 3)) & (drp1_df.d - drp1_df.r <= 5)
mtdna_cond = ((abs(abs(mtdna_df.perp_angle) - 90) < 15) | (mtdna_df.d < 3)) & (mtdna_df.d - mtdna_df.r <= 5)

In [ ]:
# Search for IDs in the area of x, y
x = slice(1200,1300)
y = slice(1450,1550)

print("DRP1", drp1_df[(drp1_df.x > x.start) & (drp1_df.x < x.stop) & (drp1_df.y > y.start) & (drp1_df.y < y.stop)].id.unique())
print("mtDNA", mtdna_df[(mtdna_df.x > x.start) & (mtdna_df.x < x.stop) & (mtdna_df.y > y.start) & (mtdna_df.y < y.stop)].id.unique())

In [ ]:
# Find DRP1 and mtDNA ids that are close by

max_t = mito_df.t.max()

drp1_ids = drp1_df[drp1_cond].drop(drp1_df[drp1_cond].groupby('id').filter(lambda x: len(x) <= max_t).index, inplace=False).id.unique()
mtdna_ids = mtdna_df[mtdna_cond].drop(mtdna_df[mtdna_cond].groupby('id').filter(lambda x: len(x) <= max_t).index, inplace=False).id.unique()

drp1_full = drp1_df[drp1_df.id.isin(drp1_ids)]
mtdna_full = mtdna_df[mtdna_df.id.isin(mtdna_ids)]

for did in drp1_ids:
    for mid in mtdna_ids:
        dis = np.linalg.norm(np.array([drp1_full[drp1_full.id == did].mx.mean(), drp1_full[drp1_full.id == did].my.mean()]) - np.array([mtdna_full[mtdna_full.id == mid].mx.mean(), mtdna_full[mtdna_full.id == mid].my.mean()]))
        if (dis < 20):
            print("DRP1 ID: ", did, "\tmtDNA ID: ", mid, "\tMean distance [px]: ", dis)

In [ ]:
show(mito,
     drp1,
     mtdna,
     mito_df,
     drp1_df,
     mtdna_df,
     mito_cmap=colors.cm_red,
     drp1_cmap=colors.cm_green,
     mtdna_cmap=colors.cm_yellow,
     frame_interval=1.0,
     spacing=3.1302974264139308E-02,
     unit="μm",
     drp1_cond=drp1_cond,
     mtdna_cond=mtdna_cond,
     window=3,
     immediate=True)